# Attention Network is All You Need

In [1]:
import pandas as pd
import numpy as np
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class FeedForward(nn.Module):
    """
    Norm Layer, follow the implementation in Layer Normalization bu Jimmy Lei Ba and al.
    (https://arxiv.org/pdf/1607.06450.pdf)
    """
    def __init__(self, d_model=512, inner_layer=2048, dropout=0.1):
        """
        Constructor of Normalization Layer
        
        Args:
            d_model: the size of the model (i.e size of vector which represents a word or sentence) (integer)
            inner_layer: the size of hidden layer between two linear transformations (integer)
        """
        super(FeedForward, self).__init__()
        self.a_1 = nn.Linear(d_model, inner_layer, bias=True)
        self.fc2 = nn.Linear(inner_layer, d_model, bias=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Forward pass for layers.
        
        Args:
            x: The input a vector of size d_model
        
        Return:
            A vector of size d_model
        
        """
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [102]:
def attention(query, key, value, mask=None, dropout=None):
    """
    Attention function compute scaled dot product attention
    
    Args:
        query: 
        keys:
        value:
    Return:
        a tensor
    """
    d_k = query.size(-1)
    
    scores = torch.matmul(query, key.T)/ np.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    scores_soft = F.softmax(scores, dim=-1)
    if dropout is not None:
        scores_soft = dropout(scores_soft)
    return torch.matmul(scores_soft, value), scores_soft

In [103]:
class NormLayer(nn.Module):
    """
    Normalization Layer resepect Jimmy Lei Ba implementation (https://arxiv.org/pdf/1607.06450.pdf)
    """
    def __init__(self, d_model=512, eps=1e-6):
        """
        Constructor Normalization Layer
        
        Args:
            d_model: the size of the model (i.e size of vector which represents a word or sentence) (integer)
            esp: 
        """
        super(NormLayer, self).__init__()
        self.b_1 = nn.Parameter(torch.zeros(d_model))
        self.a_1 = nn.Parameter(torch.randn(d_model))
        self.eps = eps

    def forward(self, x):
        """
        Forward pass for layers.
        
        Args:
            x: The input a vector of size d_model
        
        Return:
            A vector of size d_model
        
        """
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_1 * (x - mean) / (std + self.eps) + self.b_1
    
net = NormLayer()
print(net)

NormLayer()


In [104]:
tensor1 = torch.randn(3, 3)
tensor2 = torch.randn(3, 3)
tensor3 = torch.randn(3, 3)

attention(tensor1, tensor2, tensor3)

(tensor([[-1.0215,  0.5330, -0.0890],
         [-1.0048,  0.2420,  0.2985],
         [-0.8729,  0.5127, -0.1327]]),
 tensor([[0.2814, 0.4735, 0.2451],
         [0.0731, 0.2319, 0.6950],
         [0.3454, 0.3740, 0.2806]]))

In [106]:
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [107]:
class MultiHeadAttentionLayer(nn.Module):
    """
    Multi Head Attetion Network
    """
    def __init__(self, d_model=512, h=8, dropout=0.1):
        """
        Constructor for Encoder
        
        Args:
            d_model:
            h: number of parallel attention layers
        """
        super(MultiHeadAttentionLayer, self).__init__()
        self.d_k = d_model // h
        self.d_v = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        

    def forward(self, query, key, value, mask=None):
        """
        Forward pass for layers.
        
        Args:
            x: The input a vector of size d_model
        
        Return:
            A vector of size d_model
        
        """
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)
    
net = MultiHeadAttentionLayer()
print(net)

MultiHeadAttentionLayer(
  (linears): ModuleList(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)


In [188]:
class Embedding(nn.Module):
    """
    Use embedding learn to vectorize words or sentence
    """
    def __init__(self, vocab_size, d_model):
        """
        Embedding Constructor
        Args:
            vocab_size: (integer) define the size of vocab for your embedding model
            d_model: (integer) the size of model vector
        """
        super(Embedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model
        
    def forward(self, x):
        """
        Forward pass for embedding layer
        Args:
            x: (string) input
        Return:
            (Tensor) (N, d_model)
        """
        return self.embedding(x) * np.sqrt(self.d_model)

In [189]:
a = torch.range(1, 16)

/Users/pierrickrugery/miniconda3/envs/transformer/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  """Entry point for launching an IPython kernel.


In [190]:
a.view(2, -1)

tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12., 13., 14., 15., 16.]])

In [191]:
x = torch.randn(2, 3, 2)

In [192]:
torch.transpose(x, 1, 2)

tensor([[[ 1.5263,  0.8867, -0.2580],
         [-2.0673, -0.6407,  1.5761]],

        [[ 0.5027,  0.5681, -0.3630],
         [-1.1402, -0.4028,  0.0828]]])

In [193]:
class SubLayer(nn.Module):
    """
    Class sublayer perform operations between all sublayer in Encoder and Decoder
    """
    def __init__(self, d_model, dropout):
        """
        Sublayer Constructor
        Args:
            d_model: (integer) size of vector
        """
        super(SubLayer, self).__init__()
        self.norm = NormLayer(d_model=d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        """
        Forward pass for Sublayer
        Args:
            x: (Tensor) input
            sublayer: (nn.Module) a sublayer i.e MultiHeadAttentionLayer, FeedForward
        """
        return x + self.dropout(sublayer(self.norm(x)))

In [194]:
class Encoder(nn.Module):
    """
    Encoder
    """
    
    def __init__(self, layer, N):
        """
        Encoder constructor, stacks N encoder
        Args:
            layer: (nn.Module) MultiHeadAttention, FeedForward
            N: number of staked layer
        """
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = NormLayer(layer.size)
    
    def forward(self, x):
        """
        Forward pass, pass in all stacked layers
        Args:
            x: (torch.Tensor) input
        Return:
        
        """
        for layer in self.layers:
            x = layer(x)
        return self.norm(x)

In [195]:
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        """
        Encoder Layer constructor
        Args:
            size: (integer) size of input (vector to represent a word)
            self_attn: (MultiHeadAttentionLayer)
            feed_forward: (FeedForward)
        """
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.size = size
        self.sublayer = clones(SubLayer(size, dropout), 2)
    
    def forward(self, x, mask):
        """
        Forward pass, pass in MultiHeadAttention (First sublayer) then in second (FeedForward layer)
        Args:
            x: (torch.Tensor) input
        """
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [196]:
class Decoder(nn.Module):
    
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = NormLayer(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [197]:
class DecoderLayer(nn.Module):
    """
    Decoder Layer
    """
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        """
        Decoder constructor
        Args:
            size: (integer)
            self_attn: (MultiHeadAttentionLayer)
            src_attn: (torch.tensor) comes from encoder output
            feed_forward: (FeedForward)
        """
        super(DecoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.src_attn = src_attn
        self.size = size
        self.sublayer = clones(SubLayer(size, dropout), 3)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        """
        Forward pass
        Args:
            x: (torch.tensor) input
            memory: (torch.tensor) key 
            src_mask: (torch.tensor) comes from encoder
        """
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [198]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [199]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [200]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [201]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadAttentionLayer(h, d_model)
    ff = FeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embedding(d_model, src_vocab), c(position)),
        nn.Sequential(Embedding(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [202]:
tmp_model = make_model(10, 10, 2)

/Users/pierrickrugery/miniconda3/envs/transformer/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [203]:
tmp_model

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadAttentionLayer(
          (linears): ModuleList(
            (0): Linear(in_features=8, out_features=8, bias=True)
            (1): Linear(in_features=8, out_features=8, bias=True)
            (2): Linear(in_features=8, out_features=8, bias=True)
            (3): Linear(in_features=8, out_features=8, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): FeedForward(
          (a_1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (sublayer): ModuleList(
          (0): SubLayer(
            (norm): NormLayer()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): SubLayer(
            (norm): NormLayer()
            (dropout): Dropout(p=0.1, inplac